In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, 1)

Error 321, reqId -1: Error validating request.-'ca' : cause - The API interface is currently in Read-Only mode.
completed orders request timed out


<IB connected to 127.0.0.1:7497 clientId=1>

In [2]:
import pandas as pd
import time

In [5]:
contract1 = Stock('SH', 'SMART', 'USD')
ib.reqMktData(contract1, '', False, False)
t1 = ib.ticker(contract1)

Error 354, reqId 5: Requested market data is not subscribed.Delayed market data is available.Error&BEST/STK/Top&BEST/STK/Top, contract: Stock(symbol='SH', exchange='SMART', currency='USD')


In [6]:
ib.reqMarketDataType( 3 )

In [3]:
def read_two_tick_prices(stock1, stock2, minutes):
    contract1 = Stock(stock1, 'SMART', 'USD')
    ib.reqMktData(contract1, '', False, False)
    
    contract2 = Stock(stock2, 'SMART', 'USD')
    ib.reqMktData(contract2, '', False, False)

    t1 = ib.ticker(contract1)
    t2 = ib.ticker(contract2)
    ib.sleep(2)
    
    data = []
    pre_row_data = None
    
    t_start = time.time()
    t_end = t_start + 60 * minutes
    print(datetime.now())
    
    while time.time() < t_end:
        ib.sleep(0.05)
        row_data = [t1.bid, t1.bidSize,t1.ask, t1.askSize, t2.bid, t2.bidSize,  t2.ask, t2.askSize]
        if row_data != pre_row_data:
            row_time = t1.time.replace(tzinfo=timezone.utc).astimezone(tz=None)
            row = [row_time] + row_data
            data.append(row)
            pre_row_data = row_data
        else:
            pass
    
    return pd.DataFrame(data, columns=['time', 'bid1', 'bid_size1', 'ask1', 'ask_size1', 'bid2', 'bid_size2', 'ask2', 'ask_size2'])

In [7]:
%time
df = read_two_tick_prices('SH', 'SDS', 360)
df.to_csv('SH_SDS_data.csv')

Wall time: 0 ns
2020-12-03 09:49:50.199931


In [9]:
ib.disconnect()

In [8]:
df

,time,bid1,bid_size1,ask1,ask_size1,bid2,bid_size2,ask2,ask_size2
0,2020-12-03 09:49:49.496299-05:00,18.41,5057,18.42,11263,13.01,1588,13.02,2007
1,2020-12-03 09:49:50.499130-05:00,18.41,4631,18.42,11366,13.01,1576,13.02,2045
2,2020-12-03 09:50:00.518556-05:00,18.41,5562,18.42,11286,13.01,1728,13.02,1879
3,2020-12-03 09:50:10.581464-05:00,18.40,14666,18.41,244,13.01,1820,13.02,1506
4,2020-12-03 09:50:19.612368-05:00,18.41,8151,18.42,11729,13.01,1820,13.02,1506
...,...,...,...,...,...,...,...,...,...
2583,2020-12-03 15:49:09.138292-05:00,18.36,11324,18.37,6166,12.95,1992,12.96,2855
2584,2020-12-03 15:49:18.978898-05:00,18.36,10905,18.37,6233,12.95,1557,12.96,3035
2585,2020-12-03 15:49:28.996096-05:00,18.36,11367,18.37,6266,12.95,2087,12.96,2692
2586,2020-12-03 15:49:39.038233-05:00,18.36,11664,18.37,2841,12.95,2583,12.96,2015
